In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [2]:
# Note:: The make sure you use the information from your specific PostgreSQL installation
host = r'127.0.0.1' # denotes that the db in a local installation
db = r'MSDS610' # db we just created
user = r'postgres' # using the postgres user for this demo
pw = r'8751' # this is the password established during installation
port = r'5432' # default port estabalished during install
schema = r'cleaned' # schema we just created

In [3]:
# Connecting to Database
engine = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))
connection = engine.connect()

In [4]:
# Reading the data from insurance_data table
query = 'SELECT * FROM {}.insurance_data'.format('raw')
df = pd.read_sql(query, connection)
df.head()

,age,gender,bmi,children,smoker,region,medical_history,family_medical_history,exercise_frequency,occupation,coverage_level,charges
0,46,male,21.45,5,yes,southeast,Diabetes,None,Never,Blue collar,Premium,20460.307669
1,25,female,25.38,2,yes,northwest,Diabetes,High blood pressure,Occasionally,White collar,Premium,20390.899218
2,38,male,44.88,2,yes,southwest,None,High blood pressure,Occasionally,Blue collar,Premium,20204.476302
3,25,male,19.89,0,no,northwest,None,Diabetes,Rarely,White collar,Standard,11789.029843
4,49,male,38.21,3,yes,northwest,Diabetes,High blood pressure,Rarely,White collar,Standard,19268.309838


In [5]:
# Checking the data types and shape of the data
print(df.dtypes)
print(df.shape)

age                         int64
gender                     object
bmi                       float64
children                    int64
smoker                     object
region                     object
medical_history            object
family_medical_history     object
exercise_frequency         object
occupation                 object
coverage_level             object
charges                   float64
dtype: object
(1000000, 12)


In [6]:
# Checking for missing values in percentage
print(df.isnull().mean() * 100)

age                        0.0000
gender                     0.0000
bmi                        0.0000
children                   0.0000
smoker                     0.0000
region                     0.0000
medical_history           25.0762
family_medical_history    25.0404
exercise_frequency         0.0000
occupation                 0.0000
coverage_level             0.0000
charges                    0.0000
dtype: float64


In [7]:
df.medical_history.value_counts()

medical_history
Heart disease          250121
High blood pressure    249782
Diabetes               249335
Name: count, dtype: int64

In [8]:
df.family_medical_history.value_counts()

family_medical_history
Heart disease          250035
High blood pressure    249824
Diabetes               249737
Name: count, dtype: int64

In [9]:
# Replacing the missing values with 'Unknown/Not Provided'
df['medical_history'] = df['medical_history'].fillna('Unknown/Not Provided')
df['family_medical_history'] = df['family_medical_history'].fillna('Unknown/Not Provided')

# Checking for missing values in percentage
print(df.isnull().mean() * 100)

age                       0.0
gender                    0.0
bmi                       0.0
children                  0.0
smoker                    0.0
region                    0.0
medical_history           0.0
family_medical_history    0.0
exercise_frequency        0.0
occupation                0.0
coverage_level            0.0
charges                   0.0
dtype: float64


In [10]:
# Checking for duplicates
df.duplicated().sum()

0

# Creating start Schema for the cleaned data

In [11]:
categorical_cols = df.select_dtypes(include=['object']).columns
print("Categorical Columns:", categorical_cols)

Categorical Columns: Index(['gender', 'smoker', 'region', 'medical_history',
       'family_medical_history', 'exercise_frequency', 'occupation',
       'coverage_level'],
      dtype='object')


In [12]:
# Creating Dimensions tables
# Dictionary to store mappings for each categorical column
dim_tables = {}

for col in categorical_cols:
    unique_values = df[col].dropna().unique()  # Get unique categories
    dim_df = pd.DataFrame({f"{col}_id": range(1, len(unique_values) + 1), col: unique_values})
    dim_tables[col] = dim_df
    
    # Replace categorical values in the main table with their ID
    mapping_dict = dict(zip(dim_df[col], dim_df[f"{col}_id"]))
    df[col] = df[col].map(mapping_dict)


In [13]:
df.head()

,age,gender,bmi,children,smoker,region,medical_history,family_medical_history,exercise_frequency,occupation,coverage_level,charges
0,46,1,21.45,5,1,1,1,1,1,1,1,20460.307669
1,25,2,25.38,2,1,2,1,2,2,2,1,20390.899218
2,38,1,44.88,2,1,3,2,2,2,1,1,20204.476302
3,25,1,19.89,0,2,2,2,3,3,2,2,11789.029843
4,49,1,38.21,3,1,2,1,2,3,2,2,19268.309838


In [14]:
!pip install scikit-learn

In [15]:
# Normalization of the data using Min-Max Scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_scaled = df.copy()

numerical_cols = ['age', 'bmi', 'children', 'charges']
df_scaled[numerical_cols] = scaler.fit_transform(df_scaled[numerical_cols])

df_scaled = df_scaled.round(2)

# Displaying the scaled data
df_scaled.head()

,age,gender,bmi,children,smoker,region,medical_history,family_medical_history,exercise_frequency,occupation,coverage_level,charges
0,0.60,1,0.11,1.0,1,1,1,1,1,1,1,0.58
1,0.15,2,0.23,0.4,1,2,1,2,2,2,1,0.58
2,0.43,1,0.84,0.4,1,3,2,2,2,1,1,0.58
3,0.15,1,0.06,0.0,2,2,2,3,3,2,2,0.29
4,0.66,1,0.63,0.6,1,2,1,2,3,2,2,0.54


In [16]:
# creating the dimensions tables
for table_name, table_df in dim_tables.items():
    table_df.to_sql(table_name, engine, schema='cleaned', if_exists='replace', index=False)

In [17]:
df_scaled.to_sql('insurance_data_fact', con=engine, schema='cleaned', if_exists='replace', index=False, method='multi', chunksize=2000)

1000000

In [18]:
from sqlalchemy import inspect

# print the tables in the schema
inspector = inspect(engine)
print(inspector.get_table_names(schema='cleaned'))

['gender', 'smoker', 'region', 'medical_history', 'family_medical_history', 'exercise_frequency', 'occupation', 'coverage_level', 'insurance_data_fact']
